# **Character Level Multi Layer Perceptron Language Model**
This is inspired by [A Neural Probabilistic Language Model](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) paper.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
# load dataset
words = open('names.txt', 'r').read().splitlines()

In [3]:
# create lookup table for converting characters to indices
chars = sorted(list(set(''.join(words)))) # all unique characters in the dataset
stoi = {s:i+1 for i,s in enumerate(chars)} # string to index

# manually enumerate start and end token since they are not visible in the dataset
start_token = '<'
end_token = '>'
stoi[start_token] = 0
stoi[end_token] = len(stoi)

# total number of unique characters plus start and end token
chars_count = len(stoi)

# index to string
itos = {i:s for s,i in stoi.items()}

itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '<',
 27: '>'}

### Build dataset
Based on given list of words creates input tensor with sequence of characters  with length equal to 'context_length' and target tensor with next character in sequence.

In [4]:
def build_dataset(words, context_length=3):
    X, Y = [], []
    for w in words:
        context = [stoi[start_token]] * context_length # context window padded with start token
        for ch in w + end_token:
            ix = stoi[ch]
            X.append(context) # for given context ...
            Y.append(ix) # ... the next character is the target
            context = context[1:] + [ix] # crop and append - sliding window
    
    return torch.tensor(X), torch.tensor(Y)

In [5]:
# X (input) and Y (output/target/label) tensors
X, Y = build_dataset(words[:1])
print(f'X.shape = {X.shape}, X.dtype = {X.dtype}')
print(f'Y.shape = {Y.shape}, Y.dtype = {Y.dtype}')

print(f'For the first word in the dataset: {start_token + words[0] + end_token}')
for i in range(len(X)):
    print(f"X[{i}] = {X[i].tolist()} = {''.join([itos[j] for j in X[i].tolist()])}"
          f" ---and target char is--> Y[{i}] = {Y[i]} = {itos[Y[i].item()]}")

X.shape = torch.Size([5, 3]), X.dtype = torch.int64
Y.shape = torch.Size([5]), Y.dtype = torch.int64
For the first word in the dataset: <emma>
X[0] = [0, 0, 0] = <<< ---and target char is--> Y[0] = 5 = e
X[1] = [0, 0, 5] = <<e ---and target char is--> Y[1] = 13 = m
X[2] = [0, 5, 13] = <em ---and target char is--> Y[2] = 13 = m
X[3] = [5, 13, 13] = emm ---and target char is--> Y[3] = 1 = a
X[4] = [13, 13, 1] = mma ---and target char is--> Y[4] = 27 = >


### Create embedding lookup table

In [6]:
# Matrix of embeddings
C = torch.randn((chars_count, 2)) # 2-dimensional embeddings

# Example of embedding single character with index 15
# since our matrix of embeddings is the same size as the number of characters
# we can simply use the index of the character to get its embedding
example_embedding = C[15]
print(f'Embedding using index C[15]={example_embedding}')

# Alternative approach would be to one-hot encode the character and then multiply it by the embedding matrix
# this will give same result because encoded vector will have only one non-zero value equal to 1
# and this will simply act as a mask for the embedding matrix
example_embedding = F.one_hot(torch.tensor([15]), num_classes=chars_count).float() @ C
print(f'Embedding using one-hot encoding and matrix multiplication={example_embedding}')

# For rest of of notebook I will use index based approach because it is more efficient
# also thanks to python semantics we can easly retrieve embedding for whole sequence of characters
# by indexing with list or tensor of integers
example_embedding = C[torch.tensor([15, 20, 5])]
print(f'Embedding for sequence of three characters C[torch.tensor([15, 20, 5])]=\n{example_embedding}')


Embedding using index C[15]=tensor([ 0.3265, -0.4708])
Embedding using one-hot encoding and matrix multiplication=tensor([[ 0.3265, -0.4708]])
Embedding for sequence of three characters C[torch.tensor([15, 20, 5])]=
tensor([[ 0.3265, -0.4708],
        [ 0.5694,  0.0439],
        [ 0.0296, -1.2629]])


In [7]:
# What is even more mind blowing is that we can index using multidimensional tensor!
# in this exaple where X is 5x3 tensor, we will get 5x3x2 tensor with embeddings for each character in each sequence in third dimension
print(f'Shape of C[X].shape={C[X].shape}')
print(f'And C[X] =\n{C[X]}')
print(f'We can also access individual ebeddings for given sequence, for example C[X][0] =\n{C[X][0]}')
print(f'Or even embedding for individual character in sequence, for example C[X][0][0] =\n{C[X][0][0]}')

Shape of C[X].shape=torch.Size([5, 3, 2])
And C[X] =
tensor([[[-2.4789,  0.1713],
         [-2.4789,  0.1713],
         [-2.4789,  0.1713]],

        [[-2.4789,  0.1713],
         [-2.4789,  0.1713],
         [ 0.0296, -1.2629]],

        [[-2.4789,  0.1713],
         [ 0.0296, -1.2629],
         [-1.5256,  0.2417]],

        [[ 0.0296, -1.2629],
         [-1.5256,  0.2417],
         [-1.5256,  0.2417]],

        [[-1.5256,  0.2417],
         [-1.5256,  0.2417],
         [ 0.8415,  2.6921]]])
We can also access individual ebeddings for given sequence, for example C[X][0] =
tensor([[-2.4789,  0.1713],
        [-2.4789,  0.1713],
        [-2.4789,  0.1713]])
Or even embedding for individual character in sequence, for example C[X][0][0] =
tensor([-2.4789,  0.1713])


In [8]:
# Embeddings for imput sequences
X_embedded = C[X]

### MLP input and hidden layers

In [9]:
# Number of inputs for first layer
# context length * number of embeddings for each character
# in our case 3 * 2 = 6 which is the same as shape[1] * shape[2] of X_embedded
# and we will pass embeddings for each character in the sequence as input to the network
input_layer_size = X_embedded.shape[1] * X_embedded.shape[2]
print(f'input_layer_size={input_layer_size}')

# number of neurons in hidden layer
hidden_layer_size = 100
print(f'hidden_layer_size={hidden_layer_size}')

# W1 represents neural network weights between input and hidden layer
W1 = torch.randn((input_layer_size, hidden_layer_size))
print(f'W1.shape={W1.shape}')

# b1 represents bias for hidden layer
b1 = torch.randn(hidden_layer_size)
print(f'b1.shape={b1.shape}')

input_layer_size=6
hidden_layer_size=100
W1.shape=torch.Size([6, 100])
b1.shape=torch.Size([100])


In [10]:
# In order to perform matrix multiplication between input and weights we need to 
# transform input tensor which is 5x3x2 into 5x6 to fit into neural network

# This is how to access all embeddings for first character in each sequence
print(f'X_embedded[:, 0, :]=\n{X_embedded[:, 0, :]}\n')

# Knowing above we can concatenate all embeddings for each character in each sequence
print(f'torch.concat + tensor list=\n{torch.concat([X_embedded[:, i, :] for i in range(X_embedded.shape[1])], dim=1)}\n')

# We can also improve it by using torch.unbind which will return list of tensors and yield same result as for loop
print(f'torch.concat + torch.unbind=\n{torch.concat(torch.unbind(X_embedded, dim=1), dim=1)}\n')

X_embedded[:, 0, :]=
tensor([[-2.4789,  0.1713],
        [-2.4789,  0.1713],
        [-2.4789,  0.1713],
        [ 0.0296, -1.2629],
        [-1.5256,  0.2417]])

torch.concat + tensor list=
tensor([[-2.4789,  0.1713, -2.4789,  0.1713, -2.4789,  0.1713],
        [-2.4789,  0.1713, -2.4789,  0.1713,  0.0296, -1.2629],
        [-2.4789,  0.1713,  0.0296, -1.2629, -1.5256,  0.2417],
        [ 0.0296, -1.2629, -1.5256,  0.2417, -1.5256,  0.2417],
        [-1.5256,  0.2417, -1.5256,  0.2417,  0.8415,  2.6921]])

torch.concat + torch.unbind=
tensor([[-2.4789,  0.1713, -2.4789,  0.1713, -2.4789,  0.1713],
        [-2.4789,  0.1713, -2.4789,  0.1713,  0.0296, -1.2629],
        [-2.4789,  0.1713,  0.0296, -1.2629, -1.5256,  0.2417],
        [ 0.0296, -1.2629, -1.5256,  0.2417, -1.5256,  0.2417],
        [-1.5256,  0.2417, -1.5256,  0.2417,  0.8415,  2.6921]])



In [11]:
# But it turns out there is another great function in pytorch called torch.view
# Lets have some fun with it

a = torch.arange(16) # 1D tensor with 16 elements
print(f'a.shape={a.shape}, a={a}\n')
print(f'a.view(2, 8)=\n{a.view(2, 8)}\n')
print(f'a.view(4, 4)=\n{a.view(4, 4)}\n')
print(f'a.view(2, 2, 4)=\n{a.view(2, 2, 4)}\n')

a.shape=torch.Size([16]), a=tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

a.view(2, 8)=
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15]])

a.view(4, 4)=
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

a.view(2, 2, 4)=
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7]],

        [[ 8,  9, 10, 11],
         [12, 13, 14, 15]]])



In [12]:
# Knowing above we can use torch.view to transform our 5x3x2 tensor into 5x6
# Basically we just keep first dimension and merge all other dimensions into one
# Also it looks like 6 is the only possible value for second dimension because 3*2=6
# Knowing that we can use -1 for second dimension and pytorch will figure out the rest
X_embedded_flat = X_embedded.view(-1, 6)
print(f'X_embedded_flat.shape={X_embedded_flat.shape}\n')
print(f'X_embedded_flat=\n{X_embedded_flat}\n')

X_embedded_flat.shape=torch.Size([5, 6])

X_embedded_flat=
tensor([[-2.4789,  0.1713, -2.4789,  0.1713, -2.4789,  0.1713],
        [-2.4789,  0.1713, -2.4789,  0.1713,  0.0296, -1.2629],
        [-2.4789,  0.1713,  0.0296, -1.2629, -1.5256,  0.2417],
        [ 0.0296, -1.2629, -1.5256,  0.2417, -1.5256,  0.2417],
        [-1.5256,  0.2417, -1.5256,  0.2417,  0.8415,  2.6921]])



### First layer multiplication to get outputs from hidden layer
Our current desing of network is: 6 inputs which represents 2 dimensional embeddings for 3 character long sequence that go into 100 neurons in hidden layer and what we get is 100 outputs for each input sequence..

In [13]:
h = torch.tanh(X_embedded_flat @ W1 + b1) 
# + b1 is broadcasted to match shape of X_embedded_flat @ W1
# 5, 100
# 1, 100 - broadcasting will align on the right, create fake dimension on the left
# and will be copy b1 values 5 times to match shape of (X_embedded_flat @ W1) and perform elementwise addition
# and will result of adding 5x100 matrix to 5x100 matrix with copied values from b1
print(f'h.shape={h.shape}\n')

h.shape=torch.Size([5, 100])



### Output layer
Takes 100 outputs from hidden layer and produces output that represent one_hot_encoding for our characters set for each input sequence.

In [14]:
W2 = torch.randn((hidden_layer_size, chars_count))
b2 = torch.randn(chars_count)

In [15]:
# forwad pass for output layer
logits = h @ W2 + b2

# calculate output probabilities (the old way, manualy calculating softmax)
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdim=True)

# but there is faster way to calculate probabilities - use torch.nn.functional.softmax
# and this is equivalent to above calculation
probs = F.softmax(logits, dim=1)
probs.shape

torch.Size([5, 28])

In [16]:
# And what we see here is output character probabilities for second input sequence
probs[1]

tensor([7.0675e-10, 9.9818e-01, 7.5348e-15, 9.5927e-12, 6.3610e-09, 1.2919e-07,
        8.6461e-15, 5.0790e-10, 5.4979e-07, 8.1766e-08, 4.4172e-14, 6.3011e-06,
        8.3969e-07, 5.2097e-13, 1.2234e-09, 4.1684e-07, 1.0187e-07, 1.8672e-10,
        2.9145e-06, 6.9555e-13, 3.2599e-07, 1.1215e-09, 2.0190e-07, 3.3099e-11,
        9.7620e-08, 1.4056e-05, 1.3288e-09, 1.7904e-03])

### Network evaluation

In [17]:
# iterator over input sequences
it = torch.arange(X_embedded.shape[0])

# extract probabilities for target characters
target_probs = probs[it, Y]

# calculate loss as average negative log likelihood
loss = -torch.log(target_probs).mean()

print(f'Loss before training = {loss}')

Loss before training = 15.384020805358887


# Keep it short and clean without outputs
Above code show detail example of how MLP language model works with one input word.  Since I already went step by step with relevant outputs and example is clear next step is to make this code compact and efficient so it can be used for training and evaluation of language model for whole dataset.

In [18]:
# Hyperparameters
context_length = 3
embedding_size = 10
hidden_layer_size = 200

input_layer_size = context_length * embedding_size

# Build dataset for training
X, Y = build_dataset(words, context_length)

# Create embedding lookup table with size chars_count
C = torch.randn((chars_count, embedding_size), requires_grad=True)

# Create first neuron layer
W1 = torch.randn((input_layer_size, hidden_layer_size), requires_grad=True)
b1 = torch.randn(hidden_layer_size, requires_grad=True)

# Create second neuron layer
W2 = torch.randn((hidden_layer_size, chars_count), requires_grad=True)
b2 = torch.randn(chars_count, requires_grad=True)

# Parameters of the network
# high number of parameters may result in overfitting this may happen when we have more parameters than data
# in this case we may even go to loss=0 but it will not generalize well to new data, it will simply memorize training data
params = [C, W1, b1, W2, b2]
print(f'Number of parameters = {sum([p.numel() for p in params])}')
print(f'Training data set size = {X.shape[0]}')

Number of parameters = 12108
Training data set size = 228146


# Training loop
Introduce minibatch for training. When training with minibatches we don't fit all ouf our training data at once during each of training loop to calculate gradient and perform parameter update. Instead we split our training data and each epoch we take randomly selected minibatch of data and perform parameter update. This way we don't get exact gradient but we get good approximation of gradient and we can train our model much faster.

In [19]:
# Total number of epochs that took place
epochs_counter = 0

In [44]:
steps = 1000
mini_batch_size = 1000
learning_rate = 0.0001

for _ in range(steps):
    # Minibatch
    # create tensor with random indices between 0 and X.shape[0] with size mini_batch_size
    # there is much better way to do it in pytorch, but this is just for learning purposes
    # also in this case we may train on the same input sequence multiple times
    ix = torch.randint(0, X.shape[0], (mini_batch_size,))    

    # Forward pass
    X_embedded = C[X[ix]]
    h1 = torch.tanh(X_embedded.view(mini_batch_size, -1) @ W1 + b1)
    logits = h1 @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    
    # Print loss
    if epochs_counter % 100 == 0:
        print(f'Epoch {epochs_counter}, loss = {loss}')

    # Backward pass
    loss.backward(retain_graph=True)

    # Update parameters
    for p in params:
        p.data -= learning_rate * p.grad
        p.grad.data.zero_()
        
    epochs_counter += 1

Epoch 125700, loss = 2.2692623138427734
Epoch 125800, loss = 2.283143997192383
Epoch 125900, loss = 2.275668144226074
Epoch 126000, loss = 2.263411045074463
Epoch 126100, loss = 2.2675232887268066
Epoch 126200, loss = 2.35304856300354
Epoch 126300, loss = 2.290541172027588
Epoch 126400, loss = 2.3797378540039062
Epoch 126500, loss = 2.278625726699829
Epoch 126600, loss = 2.2477428913116455


### Final loss
Final loss is less than in case of bigrams (~2,454) which was expected. If we increase number of parameters results will be even better but it looks like waste of time on this concept lets move to more advanced networks ;)

In [45]:
# Final loss
h1 = torch.tanh(C[X].view(X.shape[0], -1) @ W1 + b1)
loss = F.cross_entropy(h1 @ W2 + b2, Y)
print(f'Loss on whole dataset after training for {epochs_counter} epochs = {loss}')

Loss on whole dataset after training for 126615 epochs = 2.2956297397613525


### Guide how to choose right learning rate:

1. Start with checking boundary values for your network, pick one where loss is decreasing slowly and second one when you feel like its a bit too much.

2. Use those values to create logarithmic scale for example I felt like around 0.001 and 1.2 would be good, calculate exponent for those values and create logarithmic scale with linspace:
```python
learning_rate_exponent = torch.linspace(-3, 0.2, epochs)
learning_rate = 10 ** learning_rate_exponent
```
3. Train your model with those learning rates, keep track of their corelation and plot loss vs learning rate. You should see something like this:
```python
plt.plot(lri, lossi)
plt.xlabel('Learning rate')
plt.ylabel('Loss')
````
![learning_rate_vs_loss](images/learning_rate_vs_loss.PNG)

4. Choose new boundary values where loss is lowest and repeat step 2 and 3 until you are satisfied with your learning rate.

There is one more trick to perfect model with learning rate. When you approach minimum of loss you want to decrease learning rate so you can get to the minimum. This is called learning rate annealing. You can do this by multiplying learning rate by some factor for example 0.1. You can do this manually or you can use scheduler from pytorch.

## Disclaimer why to use cross entropy vs manual average negative loss likelihood calculation

Calculate mean negative log likelihood based on softmax made of logits
using torch.nn.functional.cross_entropy() which is give same result as:
```python
counts = logits.exp() # this may cause trobles if logits are too big we will get nan, since exp for big numbers is inf
probs = counts / counts.sum(dim=1, keepdim=True)
loss = -torch.log(probs[range(len(Y)), Y]).mean()
```
- but much meory efficient because we do not create intermediate tensors
- and it is safe in case of possible big logits because internally it ofsets them by max logit value
- and much faster and... also makes backpropagation easier :)

# Sample from model

In [50]:
for _ in range(10):
    out = []
    
    context = [stoi[start_token]] * context_length
    while True:
        embedding = C[torch.tensor([context])]
        # we always consider only one sequence at a time, so we can set second dimension to -1 and pytorch will figure out the rest
        embedding_flat = embedding.view(1, -1)
        h1 = torch.tanh(embedding_flat @ W1 + b1)
        logits = h1 @ W2 + b2
        probs = F.softmax(logits, dim=1)
        
        # sample next character
        ix = torch.multinomial(probs, num_samples=1, replacement=True).item()
        context = context[1:] + [ix]
        if ix == stoi[end_token]:
            break
        out.append(itos[ix])

    print(''.join(out))

koghva
lazysan
eah
jori
zaytten
errien
wilea
ziziiah
nymanylah
abdun
